In [1]:
import requests
from bs4 import BeautifulSoup
import time

# Define Target url

In [2]:
# pubmed url
# search
# https://pubmed.ncbi.nlm.nih.gov/?term={keyword}&page={page#}
# article
# https://pubmed.ncbi.nlm.nih.gov/32265149/
base_url = 'https://pubmed.ncbi.nlm.nih.gov'
search_url = 'https://pubmed.ncbi.nlm.nih.gov/?term='
page_url = '&page='

# define scraping tags

In [3]:
# article name in search page
search_article_name_tag = 'a'
search_article_name_class = 'docsum-title'

# abstract
abstract_tag = 'div'
abstract_class = 'abstract-content selected'

keyword = 'COVID-19'

# define scraping range

In [7]:
# scrap from page start_page_num to page to_page_num
start_page_num = 101
to_page_num = 1000

# save paths
fpath = 'data/'

In [5]:
import pandas as pd

In [8]:
# start scraping
page = start_page_num

for i in range(start_page_num,to_page_num+1):
    rq = requests.get(search_url + keyword + page_url + str(i)) # get search page
    soup = BeautifulSoup(rq.text, 'lxml')
    articles = soup.find_all(search_article_name_tag, class_=search_article_name_class)

    title = []
    abstract = []
    for i in articles:
        title.append(i.text.strip()) # title
        # get abstract page
        rq = requests.get(base_url + i.get('href'))
        time.sleep(0.1)
        soup = BeautifulSoup(rq.text, 'lxml')
        # get abstract tag
        articles = soup.find_all(abstract_tag, class_='abstract-content selected')

        # check if no abstract
        if articles == []:
            abstract.append('')
            continue
        
        # get abstract raw text
        for a in articles:
            temp = a.text.split()
            temp = ' '.join(temp)

        abstract.append(temp)

    # save as csv
    d = {'title' : title, 'abstract' : abstract}
    one_page_df = pd.DataFrame(d)
    one_page_df.to_csv(fpath + str(page) + '.csv')

    page += 1
    
    time.sleep(1)